In [261]:
%load_ext autoreload
%autoreload 2
from channel_utils import Channel
from encoder_utils import Encoder
from decoder_utils import Decoder
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [262]:
L = 10 # Number of previous bits used
N = 20 # Number of output bits per input bit

In [263]:
def generate_generator_matrix():
    """
    Creates the generator matrix.
    We make sure that for each generator_polynomial (A line of the generator matrix), the first and the last bits are set to 1
    This is because we at least want b_i and b_{i-L} to be used to use the whole range L. The remaining coefficients are chosen
    at random as no method exists to optimally pick them deterministically.
    """
    generator_matrix = np.random.choice([0, 1], size=(N, L+1))
    generator_matrix[:,0] = 1
    generator_matrix[:,L] = 1
    return generator_matrix
generator_matrix = generate_generator_matrix()
print(generator_matrix)

[[1 1 0 1 0 0 0 1 0 0 1]
 [1 0 1 1 1 1 0 0 0 1 1]
 [1 1 0 0 1 1 0 0 0 0 1]
 [1 1 0 0 0 1 0 1 1 1 1]
 [1 0 0 1 0 0 1 0 1 1 1]
 [1 0 0 0 1 0 1 0 0 1 1]
 [1 1 1 1 0 1 0 1 1 1 1]
 [1 0 0 1 1 0 0 1 0 0 1]
 [1 0 0 0 1 0 0 1 1 0 1]
 [1 0 1 0 1 0 0 1 1 1 1]
 [1 0 0 1 0 1 0 1 1 1 1]
 [1 0 1 1 0 1 1 0 0 0 1]
 [1 0 1 0 0 1 1 1 1 0 1]
 [1 1 0 0 0 0 0 0 1 1 1]
 [1 1 1 0 1 1 1 0 0 0 1]
 [1 1 0 1 1 0 0 1 0 1 1]
 [1 1 0 0 0 0 1 0 0 1 1]
 [1 0 0 1 1 0 0 0 1 1 1]
 [1 1 0 1 0 0 0 1 0 0 1]
 [1 1 1 0 1 1 1 1 1 0 1]]


In [264]:
channel = Channel(changingIndex=True, verbose=False)
encoder = Encoder(input_file='test.txt', output_file='encoded.txt', generator_matrix=generator_matrix, verbose=False)
decoder = Decoder(input_file='noisy.txt', output_file='decoded.txt', generator_matrix=generator_matrix, verbose=False)

In [265]:
encoder.encode()

Original file : b'Hello world'

Bits array : [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]

Hello world
Total number of bits used : 5280


In [266]:
channel.sendFileWithOutput(inputFileName='encoded.txt', outputFileName='noisy.txt')

Channel sending with H value 2 :
[-1. -1.  0. ... -1. -1.  0.]


In [269]:
decoder.decode()

nb_trellis_sections : 88
Trellis height : 1024
Trellis width  : 89
Decoding ...
bytearray(b'Hello world')
